<a href="https://colab.research.google.com/github/EvanWAppel/work-examples/blob/main/SuretyReconciliationQ3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from google.colab import files 
import plotly.express as px
import datetime as datetime
import numpy as np
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 800)
!pip install redshift_connector
import redshift_connector
import getpass
from dateutil import parser

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 112 kB 26.0 MB/s 
     |████████████████████████████████| 128 kB 62.3 MB/s 
     |████████████████████████████████| 132 kB 56.5 MB/s 
     |████████████████████████████████| 10.3 MB 40.6 MB/s 
     |████████████████████████████████| 79 kB 8.7 MB/s 
     |████████████████████████████████| 140 kB 66.1 MB/s 
     |████████████████████████████████| 127 kB 54.6 MB/s 
     |████████████████████████████████| 105 kB 77.1 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


In [ ]:
# Credential Assignment
un = input("enter your username:")
pw = getpass.getpass("Enter your password:")

# Connect to the cluster
#https://docs.aws.amazon.com/redshift/latest/mgmt/python-connect-examples.html

# Tableau Connection
conn = redshift_connector.connect(
  host='ccos-redshift-cluster-01.clfuihuyk2so.us-east-1.redshift.amazonaws.com',
  database='tableau',
  user=un,
  password=pw,
  port=5439
)

# Create a Cursor object
cursor = conn.cursor()

# Query a table using the Cursor
# Then...
# Retrieve the query result set
# for every table you want to convert into a dataframe
# Put them in different cells so that you don't have to run the whole thing during debugging

enter your username:eappel
Enter your password:··········


In [ ]:
# CCOS transactions
cursor.execute('''select * from "ccos"."public"."transactions"  ''')
ccosTrans =  cursor.fetch_dataframe()
#print(ccosTrans.head())

In [ ]:
# CCOS Customers
cursor.execute('''select * from "ccos"."public"."clients" ''')
ccosCusts =  cursor.fetch_dataframe()
#print(ccosCusts.head())

In [ ]:
# CCOS Kiosks
cursor.execute('''select * from "ccos"."public"."kiosks" ''')
ccosKiosk =  cursor.fetch_dataframe()
#print(ccosKiosk.head())

In [ ]:
# BA transactions
cursor.execute('''select * from "ba"."public"."accountingorders"  ''')
baTrans =  cursor.fetch_dataframe()
#print(baTrans.head())

In [ ]:
# BA Customers
cursor.execute('''select * from "ba"."public"."staging-api-customers" ''')
baCusts =  cursor.fetch_dataframe()
#print(baCusts.head())

In [ ]:
# Tableau transactions
cursor.execute('''select * from "tableau"."public"."combined_transactions_live" ''')
tableau = cursor.fetch_dataframe()
tableau = tableau.sort_values(by='location_id',ascending=False)
print(tableau.head())

        platform   transaction_id                            customer_id kyc_tier     machine_id location_id kind transaction_amount transaction_currency amount_crypto crypto_currency spot_price transaction_price promo_code margin_percentage  mining_fee     completion_datetime completion_date_year completion_date_month completion_date_day completion_date_time
1139627     CCOS  T1645464937c34f  Cce7f18b8-de10-4147-b7c6-7bffcf535280        3     148055.USA    Lunknown  BUY             200.00                  USD    0.07348132             ETH    2721.78           2721.78                       0.0000  0.00000572 2022-02-21 18:08:31.033                 2022                    02                  21             8:31.033
1212586     CCOS  T1656523647fcdf  Ca20f746d-6a2f-4a10-a74b-57b5e90eb601        3  148562.USA_HQ    Lunknown  BUY              20.00                  USD    0.00100183             BTC   19963.55          19963.55   4JULYBTC            0.0000  0.00001200 2022-06-29 19:28:33.90

In [ ]:
# use LOC to pull information about locations
loc = pd.read_csv("locationsA.csv")
loc = loc.assign(lid = loc['internalId'])
loc['lid'] = loc['lid'].astype('str')

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (51,52) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
bank = pd.read_csv("bank transactions.csv")
# Bank Data cleaning
bank_cleaned = bank[['amount','bagNumberReducer','bankName','bankTransactionId','depositDate','locationIdInternal','machineId']]
# coerce to date
bank_cleaned['depositDate'] = pd.to_datetime(bank_cleaned['depositDate'])
# Only care about target date range
bank_cleaned = bank_cleaned[(bank_cleaned['depositDate']>='2022-08-01')&(bank_cleaned['depositDate']<='2022-10-31')]
# Only care about Surety
bank_cleaned = bank_cleaned[bank_cleaned['bankName']=='Surety']
# Group dates into yearmonth to easier aggregate
bank_cleaned['yearmonth'] = bank_cleaned['depositDate'].dt.to_period('M').dt.to_timestamp()
# Create aggregation to observe TARGET AMOUNTS
sums = bank_cleaned.groupby(['yearmonth'],as_index=False).agg({'amount':sum})
print(sums)
print(bank_cleaned.head())

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


   yearmonth       amount
0 2022-08-01  20676094.00
1 2022-09-01  22301590.38
2 2022-10-01  14399674.00
       amount bagNumberReducer bankName     bankTransactionId         depositDate  locationIdInternal   machineId  yearmonth
4055   2100.0       SPE3137409   Surety   10282022210090054.0 2022-10-28 07:00:00            108471.0  143136.USA 2022-10-01
4056   1800.0       SPE3137410   Surety   10282022180090056.0 2022-10-28 07:00:00            108481.0  143134.USA 2022-10-01
4057  14380.0        213008370   Surety  102820221438090048.0 2022-10-28 07:00:00            103567.0  143424.USA 2022-10-01
4058    265.0        213008369   Surety    1028202226590057.0 2022-10-28 07:00:00            101524.0  143422.USA 2022-10-01
4059    525.0       GK88708375   Surety    1028202252590058.0 2022-10-28 07:00:00            118533.0  146341.USA 2022-10-01


<ipython-input-10-430536ba2323>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bank_cleaned['depositDate'] = pd.to_datetime(bank_cleaned['depositDate'])


In [ ]:
pickups = pd.read_csv("pickups.csv")
# Pickup Data Cleaning.
pickups_cleaned = pickups[['actualDepositAmounts','amount','bagNumbers','bankDepositAmountReducer','bankReducer','bankTransactionIdsReducer','confirmAmount','dateConfirmBankDepositReducer','dateConfirmETAReducer','dateOutReducer','externalIdReducer','internalIdReducer']]
# Only Surety
pickups_cleaned = pickups_cleaned[pickups_cleaned['bankReducer']=='Surety']
# If there's a comma in the dateConfirmBankDepositReducer column then SPLIT on the comma, sort descending, then POP out the first (most recent) and call it reliable_date
pickups_cleaned['reliable_date'] = pickups_cleaned['dateConfirmBankDepositReducer'].apply(lambda x: sorted(str(x).split(','),reverse=True).pop(0) if ',' in str(x) else str(x) )
# Coerce reliable date to datetime
pickups_cleaned['reliable_date'] = pd.to_datetime(pickups_cleaned['reliable_date'])
# convert reliable date to yearmonth to aggregate easier
pickups_cleaned['yearmonth'] = pickups_cleaned['reliable_date'].dt.to_period('M').dt.to_timestamp()
# aggregate for TARGET
pickupsums = pickups_cleaned.groupby(['yearmonth'],as_index=False).agg({'amount':sum})
pickups_cleaned['bankTransactionIdsReducer'] = pickups_cleaned['bankTransactionIdsReducer'].astype('str')
#print(pickups_cleaned.head(50))
#print(pickups_cleaned.dtypes)
#print(pickupsums)


Note the difference between the totals of the bank and the totals of the pickups table. This was even more dramatic when the above LAMBDA operation didn't exist.

Are pickups getting mislabeled in CCWeb? And how?

Regardless, If I can get enough transaction records to match the above pickup amounts, that's a good start for the bank regulators and may buy us time.

In [ ]:
# Parameter Table Assembly
# Use the deposit dates from the Bank table
# Clean up the LID column
bank_cleaned['lid'] = bank_cleaned['locationIdInternal'].astype('str')
bank_cleaned['lid'] = bank_cleaned['lid'].str.replace('\.0','')
# We will start with a set of date/LID pairs
pu = bank_cleaned[['machineId','depositDate']]
#create blank parameter table
# this table will include the lid, pickup date and the most recent pickup date
# These three parameters can then be fed into a query against the transaction table (assembled below)
param = pd.DataFrame()
# for every pickup date, there is a date preceding it
# look at the list of pickup dates
# select the first item in the list
for i in pu.index:
  temp = bank_cleaned[
      # find the list of rows that have the same LID and a date less than (but not equal) to the first item, store it in temp
      (bank_cleaned['machineId']==pu['machineId'][i])
      &(bank_cleaned['depositDate']<pu['depositDate'][i])
      ]
# find the most recent date in that subset, overwrite temp
  temp = temp.groupby(['machineId'],as_index=False).agg({'depositDate':max})
# rename the most recent date from the first date as lastPickup
  temp = temp.rename(columns={'depositDate':'lastPickup'})
# join the first date, the pickup date, to the temp table in preparation for...
  temp = temp.assign(pickupdate=pu['depositDate'][i])
# appending to the parameter table
  param = param.append(temp)
# Now we should have a list of lid/pickupdate pairs matched with their most recent pickups
# Not exactly in order, but that's how it should work.
param = param.reset_index()

print(param)
print(param.dtypes)
#print(pu.head())
#print(pu.dtypes)

<ipython-input-12-dc097e872b55>:5: FutureWarning: The default value of regex will change from True to False in a future version.
  bank_cleaned['lid'] = bank_cleaned['lid'].str.replace('\.0','')


      index      machineId          lastPickup          pickupdate
0         0     143136.USA 2022-10-07 07:00:00 2022-10-28 07:00:00
1         0     143134.USA 2022-10-20 07:00:00 2022-10-28 07:00:00
2         0     143424.USA 2022-10-14 07:00:00 2022-10-28 07:00:00
3         0     143422.USA 2022-10-26 07:00:00 2022-10-28 07:00:00
4         0     146341.USA 2022-10-14 07:00:00 2022-10-28 07:00:00
...     ...            ...                 ...                 ...
5516      0  coincloud5086 2022-08-01 07:00:00 2022-08-03 07:00:00
5517      0     142012.USA 2022-08-01 07:00:00 2022-08-03 07:00:00
5518      0     143382.USA 2022-08-02 07:00:00 2022-08-03 07:00:00
5519      0     149415.USA 2022-08-01 07:00:00 2022-08-03 07:00:00
5520      0     146406.USA 2022-08-01 07:00:00 2022-08-02 07:00:00

[5521 rows x 4 columns]
index                  int64
machineId             object
lastPickup    datetime64[ns]
pickupdate    datetime64[ns]
dtype: object


In [ ]:
tableau = tableau.assign(lid = tableau['location_id'].str.replace('L',''))
parama = param.head()
print(parama)
print(tableau.head())
print(tableau.dtypes)

   index   machineId          lastPickup          pickupdate
0      0  143136.USA 2022-10-07 07:00:00 2022-10-28 07:00:00
1      0  143134.USA 2022-10-20 07:00:00 2022-10-28 07:00:00
2      0  143424.USA 2022-10-14 07:00:00 2022-10-28 07:00:00
3      0  143422.USA 2022-10-26 07:00:00 2022-10-28 07:00:00
4      0  146341.USA 2022-10-14 07:00:00 2022-10-28 07:00:00
        platform   transaction_id                            customer_id kyc_tier     machine_id location_id kind transaction_amount transaction_currency amount_crypto crypto_currency spot_price transaction_price promo_code margin_percentage  mining_fee     completion_datetime completion_date_year completion_date_month completion_date_day completion_date_time      lid
1139627     CCOS  T1645464937c34f  Cce7f18b8-de10-4147-b7c6-7bffcf535280        3     148055.USA    Lunknown  BUY             200.00                  USD    0.07348132             ETH    2721.78           2721.78                       0.0000  0.00000572 2022-02-2

In [ ]:


# Run param table against  transaction table
Surety = pd.DataFrame()
for i in param.index:
  t = pd.DataFrame()
  t = tableau[(tableau['machine_id']==param['machineId'][i])
    &(tableau['completion_datetime']>param['lastPickup'][i])
    &(tableau['completion_datetime']<=param['pickupdate'][i])
  ]
  #t = t.assign(marker=param['machineId'][i]+','+str(param['lastPickup'][i])+','+str(param['pickupdate'][i])+','+'marker'+','+str(len(t)))
  Surety = Surety.append(t)

print(Surety)

        platform     transaction_id                            customer_id kyc_tier     machine_id location_id  kind transaction_amount transaction_currency amount_crypto crypto_currency spot_price transaction_price promo_code margin_percentage  mining_fee     completion_datetime completion_date_year completion_date_month completion_date_day completion_date_time     lid
1089977     CCOS    T1666231239b042  C91427796-228e-4c6a-b78c-a51a1f60bb52        1     143136.USA     L108471   BUY            1200.00                  USD    0.05045371             BTC   19042.58          23784.17                       0.1993  0.00000772 2022-10-20 04:58:32.028                 2022                    10                  20             8:32.028  108471
1266437     CCOS    T16666513039662  Cd5e2b1fc-aa39-49df-b0b4-0d78cf218abb        1     143136.USA     L108471   BUY              50.00                  USD    0.00206902             BTC   19348.33          24166.05                       0.1993        No

In [ ]:
#Running the Parameter table against the Tableau table took about 21 minutes, but returned half of the transactions
Surety =  Surety.assign(yearmonth = Surety['completion_datetime'].dt.strftime('%Y-%m-01'))
print(Surety.head())
summer = Surety.groupby(['yearmonth'],as_index=False).agg({'transaction_amount':sum})
print(summer)

        platform   transaction_id                            customer_id kyc_tier  machine_id location_id  kind transaction_amount transaction_currency amount_crypto crypto_currency spot_price transaction_price promo_code margin_percentage  mining_fee     completion_datetime completion_date_year completion_date_month completion_date_day completion_date_time     lid   yearmonth
1089977     CCOS  T1666231239b042  C91427796-228e-4c6a-b78c-a51a1f60bb52        1  143136.USA     L108471   BUY            1200.00                  USD    0.05045371             BTC   19042.58          23784.17                       0.1993  0.00000772 2022-10-20 04:58:32.028                 2022                    10                  20             8:32.028  108471  2022-10-01
1266437     CCOS  T16666513039662  Cd5e2b1fc-aa39-49df-b0b4-0d78cf218abb        1  143136.USA     L108471   BUY              50.00                  USD    0.00206902             BTC   19348.33          24166.05                       0.1993 

In [ ]:
print(ccosTrans.head())
print(ccosKiosk.head())

                        _id  hashblockchain                         idtransaction transactiontype           status                 idatminit     transactionstartdate                 updateat currency cryptocurrencydestination     cellphone                                iduser  network       reference  locationatm.lng.double  locationatm.lat.double  tierslimits.sell  tierslimits.buy   calculateeffectivedate                  destinationaddress            effectivedate                    address.address address.zipcode address.county address.city address.state address.country  exchangeservicesspotprice promocode exchangeservice exchangerateorigin.decimal  minerfee.double cryptocurrencyprice.decimal exchangeratedestination.decimal  cryptocurrencyspotprice.double  \
0  61b7a1e052a4490008b9ada4         11478.0  ffab4f2d-b66c-4c22-be1d-4214dd4ce3af            SELL  CASH_REDEMPTION  618c75446e95ab00088ea7e1  2021-12-13 19:41:18.546  2021-12-13 19:51:15.996      USD                       BTC  

In [ ]:
c = ccosTrans[['idtransaction','status','transactiontype','idatminit','updateat','transactionamount.decimal']]
c = c[(c['transactiontype'].isin(['BUY','SELL']))&(c['status']=='COMPLETE')]
k = ccosKiosk[['_id','ccnumber','codeatm','idlocation']]
k = k.assign(idatminit = k['_id'])
c = pd.merge(c,k,on='idatminit',how='left')
c = c[['idtransaction','updateat','transactionamount.decimal','codeatm']]
c = c.rename(columns = {'updateat':'date'
                       ,'codeatm':'machineid'
                       ,'transactionamount.decimal':'amount'})
c = c.assign(platform = 'ccos')
c = c.sort_values(by='date', ascending= False)
c['amount'] = c['amount'].astype('float64')
print(c.dtypes)
print(c.head(50))

idtransaction     object
date              object
amount           float64
machineid         object
platform          object
dtype: object
                               idtransaction                     date  amount            machineid platform
100050  6065a30b-ef4a-4d28-8b28-f1d42495f7bd  2022-12-13 19:28:42.602   200.0           150034.USA     ccos
100045  9bb55fa6-b2a8-4702-a01b-a33f0cb6d88f  2022-12-13 19:28:40.743   180.0           145374.USA     ccos
100047  a10b4fcc-a284-47f7-a2a7-7e7125542c26  2022-12-13 19:28:39.919   600.0           149795.USA     ccos
100042  6ee0e092-2bde-46ad-96c6-825d0f920c63  2022-12-13 19:28:39.489    80.0           142184.USA     ccos
100040  bdb99e63-1aa1-4508-b28a-f88e241acb6c  2022-12-13 19:28:37.799   140.0           148687.USA     ccos
100046  deae4197-ae43-4cd0-b796-20f832d275d6  2022-12-13 19:28:37.504   300.0           144414.USA     ccos
100049  faf12190-f6d6-4aac-ba78-882ee6f51056  2022-12-13 19:28:37.367    35.0           143684.USA     cc

In [ ]:
b = baTrans.sort_values(by='completed at',ascending = False)
b = b[b['state']=='completed']
b = b[['btm machine name', 'id', 'amount deposited','completed at']]
b = b.rename(columns = {'btm machine name':'machineid'
                       ,'id':'idtransaction'
                       ,'amount deposited': 'amount'
                       ,'completed at':'date'})
b = b.assign(platform = 'ba')
print(b.dtypes)
print(b.head())

machineid         object
idtransaction      int64
amount           float64
date              object
platform          object
dtype: object
             machineid     idtransaction  amount                date platform
1016134  coincloud2709  1664580314413241    20.0  2022-9-30 23:48:05       ba
1016135  coincloud2709  1664580479958476   180.0  2022-9-30 23:48:05       ba
1074777  coincloud3470  1664579963351633    80.0  2022-9-30 23:48:04       ba
1015500  coincloud2023  1664579468620085   460.0  2022-9-30 23:30:36       ba
1079757  coincloud4231  1664579647262441   510.0  2022-9-30 23:30:36       ba


In [ ]:
table = pd.concat([c,b])
table = table.assign(yearmonth = pd.to_datetime(table['date']).dt.strftime("%Y-%m"))
table['date'] = pd.to_datetime(table['date'])
print(table.head())
print(table.dtypes)
t = table.groupby(['yearmonth'],as_index = False).agg({'amount':sum})
t = t[t['yearmonth'].str.startswith('2022')]
t['amount'] = t['amount'].astype('long')
print(t)

                               idtransaction                    date  amount   machineid platform yearmonth
100050  6065a30b-ef4a-4d28-8b28-f1d42495f7bd 2022-12-13 19:28:42.602   200.0  150034.USA     ccos   2022-12
100045  9bb55fa6-b2a8-4702-a01b-a33f0cb6d88f 2022-12-13 19:28:40.743   180.0  145374.USA     ccos   2022-12
100047  a10b4fcc-a284-47f7-a2a7-7e7125542c26 2022-12-13 19:28:39.919   600.0  149795.USA     ccos   2022-12
100042  6ee0e092-2bde-46ad-96c6-825d0f920c63 2022-12-13 19:28:39.489    80.0  142184.USA     ccos   2022-12
100040  bdb99e63-1aa1-4508-b28a-f88e241acb6c 2022-12-13 19:28:37.799   140.0  148687.USA     ccos   2022-12
idtransaction            object
date             datetime64[ns]
amount                  float64
machineid                object
platform                 object
yearmonth                object
dtype: object
   yearmonth    amount
42   2022-01  24431313
43   2022-02  23633037
44   2022-03  32664776
45   2022-04  34423993
46   2022-05  36331079
47   202

In [ ]:
# Run param table against  transaction table
Surety = pd.DataFrame()
for i in param.index:
  t = pd.DataFrame()
  t = table[(table['machineid']==param['machineId'][i])
    &(table['date']>param['lastPickup'][i])
    &(table['date']<=param['pickupdate'][i])
  ]
  #t = t.assign(marker=param['machineId'][i]+','+str(param['lastPickup'][i])+','+str(param['pickupdate'][i])+','+'marker'+','+str(len(t)))
  Surety = Surety.append(t)

print(Surety)

                               idtransaction                    date  amount      machineid platform yearmonth
52644   0e8c2cbc-57cc-4dd9-8253-ddfd8de39662 2022-10-25 15:29:14.874    50.0     143136.USA     ccos   2022-10
56195   a18e191b-232e-4416-aa4f-d1373cab03fa 2022-10-22 13:38:39.986   850.0     143136.USA     ccos   2022-10
60145   a8423546-e039-462c-972d-2d2693f3b042 2022-10-20 04:58:32.028  1200.0     143136.USA     ccos   2022-10
55946   233cea16-d82c-425f-96b4-d50f82743b0c 2022-10-22 21:33:14.876   100.0     143134.USA     ccos   2022-10
37472   5ae6bd34-7999-487b-8f51-fd2cbdc256ed 2022-10-22 19:31:09.735    47.0     143134.USA     ccos   2022-10
...                                      ...                     ...     ...            ...      ...       ...
932939                      1659826560471642 2022-08-06 23:25:06.000   520.0  coincloud3550       ba   2022-08
932577                      1659711237561185 2022-08-05 17:01:05.000   200.0  coincloud3550       ba   2022-08
9

In [ ]:
summer = Surety.groupby(['yearmonth'],as_index=False).agg({'amount':sum})
print(summer)

  yearmonth        amount
0   2022-08  4.630694e+06
1   2022-09  1.403452e+07
2   2022-10  1.003194e+07


YOu tried using the tableau table
you need to try to construct a table on your own of raw BA and CCOS and try it too
You also need to try machine numbers too...
Remember that the bank deposits probably have more honest machine ids than location ids


In [ ]:
# CUT AND PRINT
#Surety.to_csv('surety deposit transactions.csv')
#files.download('surety deposit transactions.csv')